#Продвинутый Python, ДЗ-4

Правила игры:

В домашке 6 задач, разбаловка указана в задании. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

В данной домашке нужно рисовать графики с помощью plotly или же Dash. Обратите внимание, что все графики и оси должны быть подписаны, графики должны выглядеть аккуратно (за это будем штрафовать)

## Данные

В этом домашнем задании вам придется оказаться на месте аналитика в бразильском маркетплейсе [Olist](https://olist.com/pt-br/). Вам необходимо исследовать данные и на их основании сделать выводы, которые помогут бизнесу расцветать!

Данные находятся тут в файле archive.zip

In [ ]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=f9e45ff67db69188faf03cec0b7f66ab2f944c8d6289d898bbd488a31842e2ee
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import wget
import plotly
import plotly.graph_objs as go
from dateutil.parser import parse
import plotly.express as px
from plotly.subplots import make_subplots

url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
filename = wget.download(url)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('./')

customers = pd.read_csv('olist_customers_dataset.csv')
location = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
translation = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

Ван дано 9 датасетов, которые содержат в себе все данные по 100 000 заказам со всей Бразилии. Чтобы облегчить вам жизнь, вот связи по этим датасетам (файл product_category_name_translation является переводом названий категорий с португальского на английский)

![](https://i.imgur.com/HRhd2Y0.png)

Ну что же, начнем!

# Часть 1. Plotly

В данной части для заданий нужно использовать Plotly для того, чтобы отобразить графики

## Задание 1 (15 баллов)

Отобразите на карте всех продавцов (в качестве точек - координаты, в качестве размера и цвета точек - число продавцов). При наведении точек должно отображаться название штата, города, а также число продавцов

*Hint: внимательно посмотрите на датасет locations. Если делать задачу максимально втупую, то у вас либо крашнется/зависнет отрисовка, либо на это нельзя будет нормально смотреть*

In [ ]:
sellers = sellers.rename(columns = {'seller_zip_code_prefix': 'geolocation_zip_code_prefix'})
merged_table = pd.merge(sellers, location, on=['geolocation_zip_code_prefix'])
merged_table = merged_table.groupby(['seller_state', 'geolocation_zip_code_prefix', 'geolocation_city']).agg({'seller_id': 'nunique', 'geolocation_lat': 'mean', 'geolocation_lng': 'mean'}).reset_index()

In [ ]:
info = merged_table[['seller_state', 'geolocation_city', 'seller_id']].copy()
info["seller_state"] = info.seller_state.apply(lambda x: 'State: ' + str(x))
info["geolocation_city"] = info.geolocation_city.apply(lambda x: ' City: ' + str(x))
info["seller_id"] = info.seller_id.apply(lambda x: ' Number of sellers: ' + str(x))

In [ ]:
def to_int_size(value):
    try:
        return int(value) + 5
    except:
        return int(value) + 5

In [ ]:
fig = go.Figure(go.Scattermapbox(lat=merged_table['geolocation_lat'],
                                 lon=merged_table['geolocation_lng'],
                                 text=info,
                                 marker=dict(colorbar=dict(title="Число продавцов"),
                                 color=merged_table['seller_id'],
                                             size=merged_table['seller_id'].map(to_int_size))))
map_center = go.layout.mapbox.Center(lat=merged_table['geolocation_lat'][10]-10, lon=merged_table['geolocation_lng'][10]-10)
fig.update_layout(mapbox_style="open-street-map", mapbox=dict(center=map_center, zoom=5))
fig.show()

## Задание 2 (15 баллов)

Отобразите на графике число заказов по дням с разбивкой по статусам. Также создайте график со средней оценкой отзыва (аггрегируйте по дням). Для обоих графиков должен быть слайдер по датам

In [ ]:
def strip_date(x):
    x = x.replace(minute=0)
    x = x.replace(second=0)
    x = x.replace(hour=0)
    return x

In [ ]:
fig = go.Figure()

orders.order_purchase_timestamp = orders.order_purchase_timestamp.apply(lambda x: parse(str(x)))
orders["order_purchase_timestamp"] = orders.order_purchase_timestamp.apply(lambda x: strip_date(x))
orders_agg = orders.groupby(['order_purchase_timestamp', 'order_status']).agg({'order_id': 'nunique'}).reset_index()
fig = px.bar(orders_agg, x=orders_agg.order_purchase_timestamp, y=orders_agg.order_id, color='order_status', labels={"order_id": "Number of orders",
                                                                                                                     "order_purchase_timestamp": "Purchase timestamp"})

fig.update_layout(
    title="Number of orders in different states",
    title_x = 0.5,
    legend=dict(x=0.9, xanchor="center", orientation="v"),
)

fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

In [ ]:
fig = go.Figure()

merged_table = pd.merge(orders, reviews[['order_id', 'review_score']], on=['order_id'])
merged_table.order_purchase_timestamp = merged_table.order_purchase_timestamp.apply(lambda x: parse(str(x)))
merged_table["order_purchase_timestamp"] = merged_table.order_purchase_timestamp.apply(lambda x: strip_date(x))
merged_table_agg = merged_table.groupby('order_purchase_timestamp').agg({'review_score': 'mean'}).reset_index()

fig.add_trace(go.Scatter(x=merged_table_agg.order_purchase_timestamp, y=merged_table_agg.review_score,line=dict(color='#7cb518')))

fig.update_layout(title_text = "Average review score", title_x=0.5)

fig.update_xaxes(title_text = "Purchase timestamp")

fig.update_yaxes(title_text = "Review score",)

fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

## Задание 3 (20 баллов)

Выберите топ-5 продавцов с более чем 20 продажами, которые чаще всего отсылают свою посылку в другие регионы и визуализируйте их отсылки

In [ ]:
merged_table = pd.merge(orders, items, on='order_id')
merged_table = pd.merge(merged_table, sellers, on='seller_id')
merged_table = pd.merge(merged_table, customers, on='customer_id')
merged_table['del_to_other_st'] = (merged_table.customer_state != merged_table.seller_state).astype(int)

merged_table = merged_table.groupby(['seller_id', 'order_id']).agg({'del_to_other_st': 'max'}).reset_index()
merged_table = merged_table.groupby('seller_id').agg({'del_to_other_st': 'sum', 'order_id': 'nunique'}).reset_index()

merged_table['share'] = merged_table['del_to_other_st'] / merged_table['order_id']
merged_table = merged_table[merged_table['order_id'] > 20]

result = merged_table.sort_values('share', ascending=False)
result = result[0:5]
idxs = np.array(result.seller_id)
idxs

array(['f62d7bb4a613ec758a03342d46b580b2',
       '679769a98cfcc6da877c8486ff1c13f3',
       '7ecd59e5e20407131822c1a68ac59c1f',
       '83deb69e889cf80f82be1dc6d5f2d486',
       '38e679b9e0064cd94c6f035707344dae'], dtype=object)

In [ ]:
sellers[sellers.seller_id.isin(idxs)]

,seller_id,geolocation_zip_code_prefix,seller_city,seller_state
289,38e679b9e0064cd94c6f035707344dae,86010,londrina,PR
291,83deb69e889cf80f82be1dc6d5f2d486,88056,florianopolis,SC
647,f62d7bb4a613ec758a03342d46b580b2,36570,vicosa,MG
816,679769a98cfcc6da877c8486ff1c13f3,35162,ipatinga,MG
2841,7ecd59e5e20407131822c1a68ac59c1f,70719,brasilia,DF


In [ ]:
top_orders = pd.merge(sellers[sellers.seller_id.isin(idxs)].rename(columns={'geolocation_zip_code_prefix': 'seller_zip_code_prefix'})[['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state']],
                      items[['order_id', 'seller_id']], on='seller_id')
top_orders = pd.merge(top_orders, orders[['order_id', 'customer_id']], on='order_id')
top_orders = pd.merge(top_orders, customers[['customer_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']], on='customer_id')
top_orders.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state,order_id,customer_id,customer_zip_code_prefix,customer_city,customer_state
0,38e679b9e0064cd94c6f035707344dae,86010,londrina,PR,047f9a218bccce8342d77f72c2240f8a,b2027e001d7cd8107cec9f7123f43930,6184,osasco,SP
1,38e679b9e0064cd94c6f035707344dae,86010,londrina,PR,1ade03eb2f7f3a76f964ee98e1a66529,b15dfeddb91c576c21ca1456d3a12f68,1308,sao paulo,SP
2,38e679b9e0064cd94c6f035707344dae,86010,londrina,PR,2d3246c68cbe85d64b652f4ff3848de2,f3de897db02b60ae259ac54f92c43d17,23098,rio de janeiro,RJ
3,38e679b9e0064cd94c6f035707344dae,86010,londrina,PR,3262303e2eba3a73ec669ac0bb7411cd,013e49bb711629177d848e409b7566c5,4578,sao paulo,SP
4,38e679b9e0064cd94c6f035707344dae,86010,londrina,PR,3630513f30286ed696e4a2e36c79ae3c,2707853be7df8de3a4c4e8c718de99ba,75930,maurilandia,GO


In [ ]:
top_sellers = pd.merge(top_orders, location[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']].rename(columns={'geolocation_zip_code_prefix': 'seller_zip_code_prefix'}),
                      on='seller_zip_code_prefix').rename(columns={'geolocation_lat': 'seller_lat',
                                                                   'geolocation_lng': 'seller_lng'})
top_sellers = top_sellers.groupby(['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state']).agg({'seller_lat': 'mean', 'seller_lng': 'mean'}).reset_index()

top_customers = pd.merge(top_orders, location[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']].rename(columns={'geolocation_zip_code_prefix': 'customer_zip_code_prefix'}),
                      on='customer_zip_code_prefix').rename(columns={'geolocation_lat': 'customer_lat',
                                                                     'geolocation_lng': 'customer_lng'})
top_customers = top_customers.groupby(['customer_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']).agg({'customer_lat': 'mean', 'customer_lng': 'mean'}).reset_index()

In [ ]:
top_orders = top_orders.merge(top_customers[['customer_id', 'customer_lat', 'customer_lng']], on='customer_id')
top_orders = top_orders.merge(top_sellers[['seller_id', 'seller_lat', 'seller_lng']], on='seller_id')
top_orders.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state,order_id,customer_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng,seller_lat,seller_lng
0,38e679b9e0064cd94c6f035707344dae,86010,londrina,PR,047f9a218bccce8342d77f72c2240f8a,b2027e001d7cd8107cec9f7123f43930,6184,osasco,SP,-23.538414,-46.807996,-23.318127,-51.157136
1,38e679b9e0064cd94c6f035707344dae,86010,londrina,PR,1ade03eb2f7f3a76f964ee98e1a66529,b15dfeddb91c576c21ca1456d3a12f68,1308,sao paulo,SP,-23.555981,-46.653311,-23.318127,-51.157136
2,38e679b9e0064cd94c6f035707344dae,86010,londrina,PR,2d3246c68cbe85d64b652f4ff3848de2,f3de897db02b60ae259ac54f92c43d17,23098,rio de janeiro,RJ,-22.870189,-43.521877,-23.318127,-51.157136
3,38e679b9e0064cd94c6f035707344dae,86010,londrina,PR,3262303e2eba3a73ec669ac0bb7411cd,013e49bb711629177d848e409b7566c5,4578,sao paulo,SP,-23.607634,-46.697252,-23.318127,-51.157136
4,38e679b9e0064cd94c6f035707344dae,86010,londrina,PR,3630513f30286ed696e4a2e36c79ae3c,2707853be7df8de3a4c4e8c718de99ba,75930,maurilandia,GO,-17.978577,-50.340164,-23.318127,-51.157136


In [ ]:
fig = go.Figure()
colors = ['#007991',
          '#EF767A',
          '#EEB868',
          '#6CC551',
          '#9055A2']
for i, seller in enumerate(idxs):
  for j in range(len(top_orders[top_orders.seller_id == seller])):
    df = top_orders[top_orders.seller_id == seller].reset_index()
    fig.add_trace(go.Scattermapbox(mode = "markers+lines",
                               hoverinfo='skip',
                               lat=[df['customer_lat'][j], df['seller_lat'][j]],
                               lon=[df['customer_lng'][j], df['seller_lng'][j]],
                  line = dict(width = 1.5,color = colors[i]), name=f"From seller number {i + 1} (state {top_orders.seller_state[i]}) to state {top_orders.customer_state[j]}"))
map_center = go.layout.mapbox.Center(lat=top_orders['seller_lat'][0]+2, lon=top_orders['seller_lng'][0]+5)
fig.update_layout(mapbox_style="open-street-map", mapbox=dict(center=map_center, zoom=5))
fig.show()

# Часть 2. Dash

В данной части нужно сделать и сдать один .py файл (можно назвать dash_task.py), который будет разворачивать страничку с нарисованными графиками (с помощью Dash)

## Задание 4 (20 баллов)

Сделайте сайт, где будет отображаться следующее:

* График, показывающий распределение по категориям продаж (то есть по тому, что продают селлеры) в штате

* График, показывающий распределение по категориям покупок (то есть по тому, что покупают пользователи) в штате

* Фильтр по штатам (должна быть возможность как выбрать отдельный штат, так и вся Бразилия в целом)

* Возможность выбрать, какие статусы продаж учитывать (чтобы можно было выбрать несколько статусов)

* Слайдер по датам (чтобы можно было выбрать даты, по которым можем смотреть результаты)


## Задание 5 (15 баллов)

Добавить карту, на котором можно будет посмотреть на количество продавцов или на количество покупателей (на него также должны работать фильтры по датам и по статусам, реализованные в прошлом занятии). Аггрегацию нужно делать по штатам. Чтобы вам помочь, вот snippet кода, который можно использовать для отображения:

*Hint: обратите внимание, что покупатели и продавцы могут быть не во всех штатах, а эжто значит, что они могут не отображаться на карте. Эту ситуацию нужно обрабатывать и поправить*

## Задание 6 (15 баллов)

Сделайте карту кликабельной: по клику на штат должен показываться сам штат (и только он), все остальные графики обновляются по выбранному штату